In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [54]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("/kaggle/input/resources/schools_complete.csv")
student_data_to_load = Path("/kaggle/input/resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

# District Summary
total_schools = len(school_data_complete['school_name'].unique())
total_students = school_data_complete['student_name'].count()
total_budget = school_data['budget'].sum()
average_math_score = school_data_complete['math_score'].mean()
average_reading_score = school_data_complete['reading_score'].mean()

passing_math = school_data_complete[school_data_complete['math_score'] >= 70]['student_name'].count()
percent_passing_math = passing_math / total_students * 100

passing_reading = school_data_complete[school_data_complete['reading_score'] >= 70]['student_name'].count()
percent_passing_reading = passing_reading / total_students * 100

passing_overall = school_data_complete[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70)]['student_name'].count()
percent_passing_overall = passing_overall / total_students * 100

district_summary_df = pd.DataFrame({
    "Total Schools": [total_schools],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [average_math_score],
    "Average Reading Score": [average_reading_score],
    "% Passing Math": [percent_passing_math],
    "% Passing Reading": [percent_passing_reading],
    "% Overall Passing": [percent_passing_overall]
})

print("District Summary:")
print(district_summary_df.head())

# School Summary
school_types = school_data.set_index('school_name')['type']
per_school_counts = school_data_complete['school_name'].value_counts()
per_school_budget = school_data_complete.groupby(['school_name']).mean()['budget']
per_student_budget = per_school_budget / per_school_counts
per_school_math = school_data_complete.groupby(['school_name']).mean()['math_score']
per_school_reading = school_data_complete.groupby(['school_name']).mean()['reading_score']
percent_passing_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['Student ID'].count() / per_school_counts * 100
percent_passing_reading = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name')['Student ID'].count() / per_school_counts * 100
percent_passing_overall = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('school_name')['Student ID'].count() / per_school_counts * 100

per_school_summary_df = pd.DataFrame({
    "School Type": school_types,
    "Total Students": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_student_budget,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    "% Passing Math": percent_passing_math,
    "% Passing Reading": percent_passing_reading,
    "% Overall Passing": percent_passing_overall
})

print("____________________________________________________________________________________")
print("Per School Summary:")
print(per_school_summary_df.head())

# Highest-Performing Schools (by % Overall Passing)
top_schools = per_school_summary_df.sort_values("% Overall Passing", ascending=False).head()

print("____________________________________________________________________________________")
print("Top Schools:")
print(top_schools.head())

#Lowest Performing Schools (by % Overall Passing)
bottom_schools = per_school_summary_df.sort_values("% Overall Passing").head()

print("____________________________________________________________________________________")
print("Lowest Performing Schools:")
print(bottom_schools.head())

# Math Scores by Grade
# Create a dictionary to store the average math scores for each grade level at each school
math_scores = {"9th": [], "10th": [], "11th": [], "12th": []}

# Iterate over each grade level and school in the DataFrame and append the average math score to the appropriate list in the dictionary
for grade in ["9th", "10th", "11th", "12th"]:
    for school in school_data_complete["school_name"].unique():
        mask = (school_data_complete["grade"] == grade) & (school_data_complete["school_name"] == school)
        math_scores[grade].append(school_data_complete.loc[mask, "math_score"].mean())

# Create a DataFrame from the dictionary
avg_math_by_school_grade = pd.DataFrame(math_scores, index=school_data_complete["school_name"].unique())


print("____________________________________________________________________________________")
print("Average Math Grade by School Grade:")
print(avg_math_by_school_grade.head())

# Reading Scores by Grade
# Create a dictionary to store the average reading scores for each grade level at each school
reading_scores = {"9th": [], "10th": [], "11th": [], "12th": []}

# Iterate over each grade level and school in the DataFrame and append the average reading score to the appropriate list in the dictionary
for grade in ["9th", "10th", "11th", "12th"]:
    for school in school_data_complete["school_name"].unique():
        mask = (school_data_complete["grade"] == grade) & (school_data_complete["school_name"] == school)
        reading_scores[grade].append(school_data_complete.loc[mask, "reading_score"].mean())

# Create a DataFrame from the dictionary
avg_reading_by_school_grade = pd.DataFrame(reading_scores, index=school_data_complete["school_name"].unique())


print("____________________________________________________________________________________")
print("Average Reading Grade by School Grade:")
print(avg_reading_by_school_grade.head())

#Scores by School Spending
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

# Create a new DataFrame with relevant columns
spending_df = per_school_summary_df[["Per Student Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]

# Add a column to the DataFrame that labels each school's spending range based on the bins defined above
spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_summary_df["Per Student Budget"], spending_bins, labels=labels)

# Group the DataFrame by spending range and calculate the mean of each column
spending_grouped = spending_df.groupby("Spending Ranges (Per Student)").mean()

# Display the resulting DataFrame
spending_grouped


# Create a DataFrame with the relevant data
school_spending_df = pd.DataFrame({
    "Average Math Score": spending_math_scores,
    "Average Reading Score": spending_reading_scores,
    "% Passing Math": spending_passing_math,
    "% Passing Reading": spending_passing_reading,
    "% Overall Passing": overall_passing_spending
})

# Remove the index name
school_spending_df.index.name = None

# Format the DataFrame
school_spending_df["Average Math Score"] = school_spending_df["Average Math Score"].map("{:.2f}".format)
school_spending_df["Average Reading Score"] = school_spending_df["Average Reading Score"].map("{:.2f}".format)
school_spending_df["% Passing Math"] = school_spending_df["% Passing Math"].map("{:.2f}%".format)
school_spending_df["% Passing Reading"] = school_spending_df["% Passing Reading"].map("{:.2f}%".format)
school_spending_df["% Overall Passing"] = school_spending_df["% Overall Passing"].map("{:.2f}%".format)

# Print the DataFrame
school_spending_df





District Summary:
   Total Schools  Total Students  Total Budget  Average Math Score  \
0             15           39170      24649428           78.985371   

   Average Reading Score  % Passing Math  % Passing Reading  % Overall Passing  
0               81.87784       74.980853          85.805463          65.172326  
____________________________________________________________________________________
Per School Summary:
                     School Type  Total Students  Total School Budget  \
Bailey High School      District            4976            3124928.0   
Cabrera High School      Charter            1858            1081356.0   
Figueroa High School    District            2949            1884411.0   
Ford High School        District            2739            1763916.0   
Griffin High School      Charter            1468             917500.0   

                      Per Student Budget  Average Math Score  \
Bailey High School                 628.0           77.048432   
Cabrera

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
<$585,83.46,83.93,93.46%,96.61%,90.37%
$585-630,81.90,83.16,87.13%,92.72%,81.42%
$630-645,78.52,81.62,73.48%,84.39%,62.86%
$645-680,77.00,81.03,66.16%,81.13%,53.53%
